In [15]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
# from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)


#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/bellybutton.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Samples_Metadata = Base.classes.sample_metadata
Samples = Base.classes.samples

C:\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [ ]:
@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")


@app.route("/names")
def names():
    """Return a list of sample names."""

    # Use Pandas to perform the sql query
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Return a list of the column names (sample names)
    return jsonify(list(df.columns)[2:])


@app.route("/metadata/<sample>")
def sample_metadata(sample):
    """Return the MetaData for a given sample."""
    sel = [
        Samples_Metadata.sample,
        Samples_Metadata.ETHNICITY,
        Samples_Metadata.GENDER,
        Samples_Metadata.AGE,
        Samples_Metadata.LOCATION,
        Samples_Metadata.BBTYPE,
        Samples_Metadata.WFREQ,
    ]

    results = db.session.query(*sel).filter(Samples_Metadata.sample == sample).all()

    # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata["sample"] = result[0]
        sample_metadata["ETHNICITY"] = result[1]
        sample_metadata["GENDER"] = result[2]
        sample_metadata["AGE"] = result[3]
        sample_metadata["LOCATION"] = result[4]
        sample_metadata["BBTYPE"] = result[5]
        sample_metadata["WFREQ"] = result[6]

    print(sample_metadata)
    return jsonify(sample_metadata)


@app.route("/samples/<sample>")
def samples(sample):
    """Return `otu_ids`, `otu_labels`,and `sample_values`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
    # Format the data to send as json
    data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }
    return jsonify(data)


if __name__ == "__main__":
    app.run(debug=True)

In [29]:
def samples(sample):
    """Return `otu_ids`, `otu_labels`,and `sample_values`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
    sample_data.sort_values(by=sample,ascending=False, inplace=True)
    print(sample_data)
    # Format the data to send as json
    data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }
    return (data)

In [30]:
samples('940')

      otu_id                                          otu_label  940
1166    1167  Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...  163
2858    2859  Bacteria;Firmicutes;Clostridia;Clostridiales;I...  126
481      482                                           Bacteria  113
2263    2264  Bacteria;Firmicutes;Clostridia;Clostridiales;I...   78
40        41                                           Bacteria   71
1188    1189  Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...   51
351      352                                           Bacteria   50
188      189                                           Bacteria   47
2317    2318  Bacteria;Firmicutes;Clostridia;Clostridiales;I...   40
1976    1977       Bacteria;Firmicutes;Clostridia;Clostridiales   40
3449    3450  Bacteria;Proteobacteria;Epsilonproteobacteria;...   37
873      874  Bacteria;Actinobacteria;Actinobacteria;Actinom...   36
1958    1959                     Bacteria;Firmicutes;Clostridia   30
2190    2191  Bacteria;Firmicutes;

{'otu_ids': [1167,
  2859,
  482,
  2264,
  41,
  1189,
  352,
  189,
  2318,
  1977,
  3450,
  874,
  1959,
  2191,
  1950,
  2077,
  2275,
  944,
  2184,
  2244,
  2024,
  2419,
  2811,
  165,
  2782,
  2247,
  2011,
  2396,
  830,
  2964,
  1795,
  2722,
  307,
  2178,
  2908,
  1193,
  2167,
  1208,
  2039,
  1274,
  2739,
  2737,
  1314,
  1962,
  2186,
  2335,
  2936,
  907,
  833,
  2483,
  2475,
  2491,
  2291,
  159,
  2571,
  2350,
  2342,
  2546,
  725,
  170,
  1505,
  513,
  259,
  1169,
  258,
  1232,
  1497,
  1498,
  1503,
  412,
  2235,
  1960,
  1968,
  121,
  2065,
  340,
  2110,
  2188,
  357,
  342],
 'sample_values': [163,
  126,
  113,
  78,
  71,
  51,
  50,
  47,
  40,
  40,
  37,
  36,
  30,
  28,
  25,
  23,
  22,
  19,
  19,
  14,
  13,
  13,
  13,
  12,
  12,
  11,
  11,
  11,
  10,
  10,
  10,
  8,
  7,
  7,
  7,
  6,
  5,
  5,
  5,
  4,
  4,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
